# 데이터 집계와 그룹 연산

## 10.1 GroupBy 메카닉

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.DataFrame({'key1': ['a', 'a', 'b', 'b', 'a'], 'key2': ['one', 'two', 'one', 'two', 'one'], 'data1': np.random.randn(5), 'data2': np.random.randn(5)})

In [3]:
df

,key1,key2,data1,data2
0,a,one,0.230635,0.783628
1,a,two,-0.362039,-0.872695
2,b,one,0.427792,0.005288
3,b,two,-0.536299,0.923150
4,a,one,0.171935,-1.427296


In [4]:
grouped = df.groupby(df['key1'])

In [5]:
grouped.mean()

,data1,data2
key1,,
a,0.013510,-0.505455
b,-0.054254,0.464219


In [6]:
grouped.describe()

,count,mean,std,min,25%,50%,75%,max
key1,,,,,,,,
a,3.0,-1.176203,0.731679,-1.796121,-1.579743,-1.363366,-0.866244,-0.369122
b,2.0,-0.409930,0.618853,-0.847524,-0.628727,-0.409930,-0.191132,0.027665


In [7]:
grouped.mean()

key1
a   -1.176203
b   -0.409930
Name: data1, dtype: float64

In [8]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()

In [9]:
means

key1  key2
a     one    -1.082622
      two    -1.363366
b     one     0.027665
      two    -0.847524
Name: data1, dtype: float64

In [10]:
means.unstack()

key2,one,two
key1,,
a,-1.082622,-1.363366
b,0.027665,-0.847524


In [11]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])

In [12]:
years = np.array([2005, 2005, 2006, 2005, 2006])

In [13]:
df['data1'].groupby([states, years]).mean()

California  2005   -1.363366
            2006    0.027665
Ohio        2005   -0.608323
            2006   -1.796121
Name: data1, dtype: float64

In [14]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,-1.176203,0.473125
b,-0.409930,-0.764081


In [15]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one  -1.082622  0.640813
     two  -1.363366  0.137750
b    one   0.027665 -0.627379
     two  -0.847524 -0.900783

In [16]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### 10.1.1 그룹 간 순회하기

In [19]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -0.369122  1.474652
1    a  two -1.363366  0.137750
4    a  one -1.796121 -0.193027
b
  key1 key2     data1     data2
2    b  one  0.027665 -0.627379
3    b  two -0.847524 -0.900783


In [21]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print(k1, k2)
    print(group)

a one
  key1 key2     data1     data2
0    a  one -0.369122  1.474652
4    a  one -1.796121 -0.193027
a two
  key1 key2     data1    data2
1    a  two -1.363366  0.13775
b one
  key1 key2     data1     data2
2    b  one  0.027665 -0.627379
b two
  key1 key2     data1     data2
3    b  two -0.847524 -0.900783


In [22]:
pieces = dict(list(df.groupby('key1')))

In [23]:
pieces['b']

,key1,key2,data1,data2
2,b,one,0.027665,-0.627379
3,b,two,-0.847524,-0.900783


In [24]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [25]:
grouped = df.groupby(df.dtypes, axis=1)

In [26]:
grouped

In [27]:
for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0 -0.369122  1.474652
1 -1.363366  0.137750
2  0.027665 -0.627379
3 -0.847524 -0.900783
4 -1.796121 -0.193027
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### 10.1.2 컬럼이나 컬럼의 일부만 선택하기

In [29]:
df.groupby('key1')['data1']

In [30]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one   0.640813
     two   0.137750
b    one  -0.627379
     two  -0.900783

In [31]:
s_grouped = df.groupby(['key1', 'key2'])['data2']

In [32]:
s_grouped

In [33]:
s_grouped.mean()

key1  key2
a     one     0.640813
      two     0.137750
b     one    -0.627379
      two    -0.900783
Name: data2, dtype: float64

### 10.1.3 사전과 Series에서 그룹핑하기

In [39]:
people = pd.DataFrame(np.random.randn(5,5), columns=['a', 'b', 'c', 'd', 'e'], index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])

In [40]:
people.iloc[2:3, [1,2]] = np.nan

In [41]:
people

,a,b,c,d,e
Joe,1.102468,-0.091701,-0.136730,0.172954,0.122298
Steve,-2.002582,-0.914207,-0.909912,-0.395190,0.290595
Wes,0.434959,NaN,NaN,-0.026837,-0.660615
Jim,-0.597132,-1.075223,-0.543470,0.393490,0.410913
Travis,1.271624,0.300006,1.018990,-0.436019,1.591871


In [42]:
mapping = {'a':'red', 'b':'red', 'c':'blue', 'd':'blue', 'e':'red', 'f':'orange'}

In [43]:
by_column = people.groupby(mapping, axis=1)

In [44]:
by_column.sum()

,blue,red
Joe,0.036224,1.133065
Steve,-1.305102,-2.626194
Wes,-0.026837,-0.225657
Jim,-0.149980,-1.261443
Travis,0.582971,3.163501


In [45]:
map_series = pd.Series(mapping)

In [46]:
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [47]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### 10.1.4 함수로 그룹핑하기

In [48]:
people.groupby(len).sum()

,a,b,c,d,e
3,0.940295,-1.166924,-0.680200,0.539607,-0.127405
5,-2.002582,-0.914207,-0.909912,-0.395190,0.290595
6,1.271624,0.300006,1.018990,-0.436019,1.591871


In [49]:
key_list = ['one', 'one', 'one', 'two', 'two']

In [50]:
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one  0.434959 -0.091701 -0.136730 -0.026837 -0.660615
  two -0.597132 -1.075223 -0.543470  0.393490  0.410913
5 one -2.002582 -0.914207 -0.909912 -0.395190  0.290595
6 two  1.271624  0.300006  1.018990 -0.436019  1.591871

### 10.1.5 색인 단계로 그룹핑하기

In [52]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'], [1,3,5,1,3]], names=['cty', 'tenor'])

In [53]:
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)

In [54]:
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0     -0.907526  0.330143  2.220737 -0.369577  0.288739
1     -0.014288  0.579239  0.756813 -1.919925  0.500321
2      0.964130  1.005097 -0.862401  0.066956 -0.593506
3      0.796723 -0.182209  1.345309 -0.312400 -0.533801

In [56]:
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


### 10.2 데이터 집계

In [58]:
df

,key1,key2,data1,data2
0,a,one,-0.369122,1.474652
1,a,two,-1.363366,0.137750
2,b,one,0.027665,-0.627379
3,b,two,-0.847524,-0.900783
4,a,one,-1.796121,-0.193027


In [59]:
grouped = df.groupby('key1')

In [60]:
grouped['data1'].quantile(0.9)

key1
a   -0.567971
b   -0.059854
Name: data1, dtype: float64

In [61]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [62]:
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,1.426999,1.667679
b,0.875190,0.273404


In [63]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -1.176203  0.731679 -1.796121 -1.579743 -1.363366 -0.866244   
b      2.0 -0.409930  0.618853 -0.847524 -0.628727 -0.409930 -0.191132   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a    -0.369122   3.0  0.473125  0.882975 -0.193027 -0.027638  0.137750   
b     0.027665   2.0 -0.764081  0.193326 -0.900783 -0.832432 -0.764081   

                          
           75%       max  
key1                      
a     0.806201  1.474652  
b    -0.695730 -0.627379

### 10.2.1 컬럼에 여러 가지 함수 적용하기

In [64]:
tips = pd.read_csv('examples/tips.csv')

In [65]:
tips['tip_pct'] = tips['tip'] / tips['total_bill']

In [66]:
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [67]:
grouped = tips.groupby(['day', 'smoker'])

In [68]:
grouped_pct = grouped['tip_pct']

In [69]:
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [70]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [71]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [74]:
functions = ('count', 'mean', 'max')

In [75]:
result = grouped['tip_pct', 'total_bill'].agg(functions)

<ipython-input-75-426117f1e146>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  result = grouped['tip_pct', 'total_bill'].agg(functions)


In [76]:
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [77]:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [78]:
ftuple = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]

In [79]:
grouped['tip_pct', 'total_bill'].agg(ftuple)

<ipython-input-79-20aa05fb1118>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  grouped['tip_pct', 'total_bill'].agg(ftuple)


tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

In [81]:
grouped.agg({'tip':np.max, 'size':'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [82]:
grouped.agg({'tip_pct':['min', 'max', 'mean', 'std'], 'size':'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

### 10.2.2 색인되지 않은 형태로 집계된 데이터 반환하기

In [83]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


### 10.3 Apply: 일반적인 분리-적용-병합

In [84]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]

In [91]:
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [88]:
tips.sort_values(by='tip_pct')

,total_bill,tip,smoker,day,time,size,tip_pct
237,32.83,1.17,Yes,Sat,Dinner,2,0.035638
102,44.30,2.50,Yes,Sat,Dinner,3,0.056433
57,26.41,1.50,No,Sat,Dinner,2,0.056797
0,16.99,1.01,No,Sun,Dinner,2,0.059447
187,30.46,2.00,Yes,Sun,Dinner,5,0.065660
...,...,...,...,...,...,...,...
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667


In [92]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [93]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [94]:
result = tips.groupby('smoker')['tip_pct'].describe()

In [95]:
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [97]:
result.unstack()

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

### 10.3.1 그룹 색인 생략하기

In [100]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


### 10.3.2 변위치 분석과 버킷 분석

In [101]:
frame = pd.DataFrame({'data1': np.random.randn(1000), 'data2': np.random.randn(1000)})

In [102]:
quartiles = pd.cut(frame.data1, 4)

In [104]:
quartiles[:10]

0    (-1.612, -0.0431]
1    (-1.612, -0.0431]
2    (-1.612, -0.0431]
3     (-0.0431, 1.526]
4     (-0.0431, 1.526]
5    (-1.612, -0.0431]
6    (-1.612, -0.0431]
7     (-0.0431, 1.526]
8    (-1.612, -0.0431]
9    (-1.612, -0.0431]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.187, -1.612] < (-1.612, -0.0431] < (-0.0431, 1.526] < (1.526, 3.095]]

In [108]:
def get_stats(group):
    return {'min':group.min(), 'max':group.max(), 'count':group.count(), 'mean':group.mean()}

In [109]:
grouped = frame.data2.groupby(quartiles)

In [110]:
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-3.187, -1.612]",-1.725303,2.054511,45.0,0.318342
"(-1.612, -0.0431]",-2.807240,2.971163,473.0,0.037678
"(-0.0431, 1.526]",-3.534253,3.444301,425.0,0.023974
"(1.526, 3.095]",-1.175501,2.302728,57.0,0.271987


In [111]:
grouping = pd.qcut(frame.data1, 10, labels=False)

In [112]:
grouped = frame.data2.groupby(grouping)

In [113]:
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
0,-2.364401,2.971163,100.0,0.226090
1,-2.022596,2.418617,100.0,0.005039
2,-2.807240,2.491130,100.0,0.011803
3,-2.574691,2.417940,100.0,0.136000
4,-1.657287,1.918456,100.0,-0.014544
5,-2.979781,2.988461,100.0,-0.042838
6,-2.316321,3.444301,100.0,0.117222
7,-2.147613,2.330715,100.0,0.075822
8,-3.534253,2.241408,100.0,-0.032726


### 10.3.3 예제: 그룹에 따른 값으로 결측치 채우기

In [117]:
s = pd.Series(np.random.randn(6))

In [118]:
s[::2] = np.nan

In [119]:
s

0         NaN
1    0.440680
2         NaN
3    0.263594
4         NaN
5    0.638696
dtype: float64

In [120]:
s.fillna(s.mean())

0    0.447656
1    0.440680
2    0.447656
3    0.263594
4    0.447656
5    0.638696
dtype: float64

In [121]:
states = ['Ohio', 'New York', 'Vermont', 'Florida', 'Oregon', 'Nevada', 'California', 'Idaho']

In [122]:
group_key = ['East'] * 4 + ['West'] * 4

In [123]:
data = pd.Series(np.random.randn(8), index=states)

In [124]:
data

Ohio         -1.943588
New York      0.258152
Vermont       0.627658
Florida       0.046602
Oregon        1.285148
Nevada       -1.569387
California   -2.366782
Idaho         0.146516
dtype: float64

In [127]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan

In [128]:
data

Ohio         -1.943588
New York      0.258152
Vermont            NaN
Florida       0.046602
Oregon        1.285148
Nevada             NaN
California   -2.366782
Idaho              NaN
dtype: float64

In [129]:
data.groupby(group_key).mean()

East   -0.546278
West   -0.540817
dtype: float64

In [130]:
fill_mean = lambda g: g.fillna(g.mean())

In [131]:
data.groupby(group_key).apply(fill_mean)

Ohio         -1.943588
New York      0.258152
Vermont      -0.546278
Florida       0.046602
Oregon        1.285148
Nevada       -0.540817
California   -2.366782
Idaho        -0.540817
dtype: float64

In [132]:
fill_values = {'East': 0.5, 'West': -1}

In [133]:
fill_func = lambda g: g.fillna(fill_values[g.name])

In [134]:
data.groupby(group_key).apply(fill_func)

Ohio         -1.943588
New York      0.258152
Vermont       0.500000
Florida       0.046602
Oregon        1.285148
Nevada       -1.000000
California   -2.366782
Idaho        -1.000000
dtype: float64

### 10.3.4 예제: 랜덤 표본과 순열

In [136]:
suits = ['H', 'S', 'C', 'D']

In [137]:
card_val = (list(range(1, 11))+ [10] * 3) * 4

In [138]:
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']

In [139]:
cards = []

In [141]:
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)

In [142]:
deck = pd.Series(card_val, index=cards)

In [144]:
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [145]:
def draw(deck, n=5):
    return deck.sample(n)

In [146]:
draw(deck)

JC    10
6D     6
5C     5
9S     9
5H     5
dtype: int64

In [147]:
get_suit = lambda card: card[-1]

In [148]:
deck.groupby(get_suit).apply(draw, n=2)

C  JC     10
   9C      9
D  7D      7
   10D    10
H  9H      9
   4H      4
S  4S      4
   5S      5
dtype: int64

In [149]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

JC     10
AC      1
3D      3
4D      4
JH     10
2H      2
10S    10
KS     10
dtype: int64

### 10.3.5 예제: 그룹 가중 평균과 상관관계

In [150]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a', 'b', 'b', 'b', 'b'], 'data': np.random.randn(8), 'weights': np.random.rand(8)})

In [151]:
df

,category,data,weights
0,a,1.147321,0.157813
1,a,-0.613352,0.338592
2,a,0.015503,0.994314
3,a,-0.271402,0.574116
4,b,0.454933,0.689479
5,b,0.788070,0.637220
6,b,-1.112281,0.365596
7,b,-0.690121,0.846344


In [153]:
grouped = df.groupby('category')

In [154]:
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])

In [155]:
grouped.apply(get_wavg)

category
a   -0.080886
b   -0.068889
dtype: float64

In [156]:
close_px = pd.read_csv('examples/stock_px_2.csv', parse_dates=True, index_col=0)

In [157]:
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [158]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [166]:
spx_corr = lambda x: x.corrwith(x['SPX'])

In [167]:
rets = close_px.pct_change().dropna()

In [168]:
get_year = lambda x: x.year

In [169]:
by_year = rets.groupby(get_year)

In [170]:
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [171]:
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

### 10.3.6 예제: 그룹상의 선형회귀

In [175]:
import statsmodels.api as sm

def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

In [176]:
by_year.apply(regress, 'AAPL', ['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


## 10.4 피벗테이블과 교차일람표

In [177]:
tips.pivot_table(index=['day', 'smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [180]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'], columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [181]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'], columns='smoker', margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [182]:
tips.pivot_table('tip_pct', index=['time', 'smoker'], columns='day', aggfunc=len, margins=True)

day             Fri   Sat   Sun  Thur    All
time   smoker                               
Dinner No       3.0  45.0  57.0   1.0  106.0
       Yes      9.0  42.0  19.0   NaN   70.0
Lunch  No       1.0   NaN   NaN  44.0   45.0
       Yes      6.0   NaN   NaN  17.0   23.0
All            19.0  87.0  76.0  62.0  244.0

In [184]:
tips.pivot_table('tip_pct', index=['time', 'size', 'smoker'], columns='day', aggfunc='mean', fill_value=0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

### 10.4.1 교차일람표

In [187]:
from io import StringIO
data = """\
Sample  Nationality  Handedness
1   USA  Right-handed
2   Japan    Left-handed
3   USA  Right-handed
4   Japan    Right-handed
5   Japan    Left-handed
6   Japan    Right-handed
7   USA  Right-handed
8   USA  Left-handed
9   Japan    Right-handed
10  USA  Right-handed"""
data = pd.read_table(StringIO(data), sep='\s+')

In [189]:
pd.crosstab(data.Nationality, data.Handedness, margins=True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


In [190]:
pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244